# 2 level full factorial DOE creator

*Updated 8/25/24*

Skip to block#6 to enter your factors and levels, then run from the beginning

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import itertools
import math
import statsmodels.api as sm

In [ ]:
def number_to_binary(number, length):
  """Converts a number to its binary representation with a specified length.

  Args:
    number: The number to convert to binary.
    length: The desired length of the binary string.

  Returns:
    A string representing the binary form of the number with the given length.
  """
  binary_string = format(number, '0' + str(length) + 'b')
  return binary_string

In [ ]:
def linear_mapping(vector,old_range,new_range):
  return (vector-old_range[0])/(old_range[1]-old_range[0])*(new_range[1]-new_range[0])+new_range[0]

In [ ]:
def create_2level_full_factorial_design(factor_num, replication,randomize,center_point):
    # Create a 2-level full factorial design
    # factor_num, replication are integers, return an array of coded experiment plan
    combo_num=2**factor_num
    combo=np.arange(1,combo_num+1)
    total_num=combo_num*factor_num
    if center_point:
      total_num=total_num+factor_num+1
    plan=np.zeros((total_num,factor_num+1))
    for j in range (replication):
      if randomize:
        combo=np.random.permutation(combo)
      design=np.zeros((combo_num,factor_num))
      for i in range(combo_num):
        number =combo[i]-1
        binary_string = number_to_binary(number, factor_num)
        run = [int(bit) for bit in binary_string]
        design[i,:]=run
      design = design*2-1
      merged_array = np.column_stack((combo, design))
      if center_point:
        plan[j*combo_num+1*(j+1):(j+1)*combo_num+1*(j+1),:]=merged_array
      else:
        plan[j*combo_num:(j+1)*combo_num,:]=merged_array
    return plan

In [ ]:
def map_factors_to_ranges(factors, factor_ranges,plan_arr):
  arr=plan_arr[:,1:]
  mapped_arr=np.zeros((len(arr),len(factors)))
  #print(arr)
  for i in range(len(factors)):
    mapped_arr[:,i]=linear_mapping(arr[:,i],[-1,1],factor_ranges[i])
  #print(mapped_arr)
  plan_arr=np.column_stack((plan_arr[:,0],mapped_arr))
  return plan_arr

This is the place you input the factors and their levels for the design.  It creates an Excel spreadsheet for a coded 2-level full factorial design.

In [ ]:
factors = ['A', 'B', 'C']
factor_ranges=[[22.9,94.1],[13,23],[40.6,43]]
replication=3
center_point=True
randomize=True
factor_num=len(factors)
plan_arr = create_2level_full_factorial_design(factor_num, replication,randomize,center_point)
print(plan_arr)
label_list=['combo#']+factors
print(label_list)
df = pd.DataFrame(plan_arr, columns=label_list)
df.to_excel('full_factorial_design_catapult_coded.xlsx', index=True)


[[ 0.  0.  0.  0.]
 [ 7.  1.  1. -1.]
 [ 8.  1.  1.  1.]
 [ 3. -1.  1. -1.]
 [ 5.  1. -1. -1.]
 [ 4. -1.  1.  1.]
 [ 6.  1. -1.  1.]
 [ 1. -1. -1. -1.]
 [ 2. -1. -1.  1.]
 [ 0.  0.  0.  0.]
 [ 3. -1.  1. -1.]
 [ 7.  1.  1. -1.]
 [ 2. -1. -1.  1.]
 [ 8.  1.  1.  1.]
 [ 6.  1. -1.  1.]
 [ 5.  1. -1. -1.]
 [ 1. -1. -1. -1.]
 [ 4. -1.  1.  1.]
 [ 0.  0.  0.  0.]
 [ 4. -1.  1.  1.]
 [ 6.  1. -1.  1.]
 [ 7.  1.  1. -1.]
 [ 5.  1. -1. -1.]
 [ 2. -1. -1.  1.]
 [ 1. -1. -1. -1.]
 [ 8.  1.  1.  1.]
 [ 3. -1.  1. -1.]
 [ 0.  0.  0.  0.]]
['combo#', 'A', 'B', 'C']


If you want to run the experiment with real levels (not coded ones), run the following code to create another Excel spreadsheet.

In [ ]:
real_arr=map_factors_to_ranges(factors, factor_ranges,plan_arr)
print(real_arr)
label_list=['combo#']+factors
print(label_list)
df = pd.DataFrame(real_arr, columns=label_list)
df.to_excel('full_factorial_design_catapult_real.xlsx', index=True)

[[ 0.  58.5 18.  41.8]
 [ 7.  94.1 23.  40.6]
 [ 8.  94.1 23.  43. ]
 [ 3.  22.9 23.  40.6]
 [ 5.  94.1 13.  40.6]
 [ 4.  22.9 23.  43. ]
 [ 6.  94.1 13.  43. ]
 [ 1.  22.9 13.  40.6]
 [ 2.  22.9 13.  43. ]
 [ 0.  58.5 18.  41.8]
 [ 3.  22.9 23.  40.6]
 [ 7.  94.1 23.  40.6]
 [ 2.  22.9 13.  43. ]
 [ 8.  94.1 23.  43. ]
 [ 6.  94.1 13.  43. ]
 [ 5.  94.1 13.  40.6]
 [ 1.  22.9 13.  40.6]
 [ 4.  22.9 23.  43. ]
 [ 0.  58.5 18.  41.8]
 [ 4.  22.9 23.  43. ]
 [ 6.  94.1 13.  43. ]
 [ 7.  94.1 23.  40.6]
 [ 5.  94.1 13.  40.6]
 [ 2.  22.9 13.  43. ]
 [ 1.  22.9 13.  40.6]
 [ 8.  94.1 23.  43. ]
 [ 3.  22.9 23.  40.6]
 [ 0.  58.5 18.  41.8]]
['combo#', 'A', 'B', 'C']
